##PersonText un corpus con base en texto orientado a personalidad.
**Director de Tesis**: Dr. Ramón Zatarain Cabada<br>
**Asesora interna**: Dra. María Lucía Barrón Estrada<br>
**Investigadores**: <br>
  M.C. Víctor Manuel Bátiz Beltrán<br>
  Gerardo Ángel Ruiz Beltrán

  Trabajando en ella: Gerardo/Víctor

### Enfoque
En esta implementación estaremos explorando la clasificación de los rasgos de personalidad para crear un modelo que sea capaz de determinar la presencia o no de un rasgo en particular.

### Primeros pasos
Instalamos e importamos las bibliotecas que estaremos utilizando.

In [1]:
import re
import matplotlib.pyplot as plt
import string
from nltk.corpus import stopwords
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk import SnowballStemmer
import unicodedata
from collections import Counter
from wordcloud import WordCloud
from gensim.utils import simple_preprocess
import gensim
from sklearn.model_selection import train_test_split
import spacy
import pickle
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt 
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import keras
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
print('Listo')

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Listo


## 1. Cargar los datos

### Descargando el corpus Mypersonality

In [2]:
!wget --no-check-certificate \
    "https://person-app-itc.web.app/corpus/corpus_mypersonality.csv" \
    -O "mypersonality_final.csv"

--2023-03-11 01:23:36--  https://person-app-itc.web.app/corpus/corpus_mypersonality.csv
Resolving person-app-itc.web.app (person-app-itc.web.app)... 199.36.158.100, 2620:0:890::100
Connecting to person-app-itc.web.app (person-app-itc.web.app)|199.36.158.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2190737 (2.1M) [text/csv]
Saving to: ‘mypersonality_final.csv’

mypersonality_final 100%[===================>]   2.09M  --.-KB/s    in 0.06s   

2023-03-11 01:23:36 (33.2 MB/s) - ‘mypersonality_final.csv’ saved [2190737/2190737]



In [3]:
datos = pd.read_csv("mypersonality_final.csv",sep=",", encoding='cp1252')

### Exploración de los datos

In [4]:
datos.head()

,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,cAGR,cCON,cOPN,DATE,NETWORKSIZE,BETWEENNESS,NBETWEENNESS,DENSITY,BROKERAGE,NBROKERAGE,TRANSITIVITY
0,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,06/19/09 03:21 PM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
1,b7b7764cfa1c523e4e93ab2a79a946c4,is so sleepy it's not even funny that's she ca...,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,07/02/09 08:41 AM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
2,b7b7764cfa1c523e4e93ab2a79a946c4,is sore and wants the knot of muscles at the b...,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,06/15/09 01:15 PM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
3,b7b7764cfa1c523e4e93ab2a79a946c4,likes how the day sounds in this new song.,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,06/22/09 04:48 AM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
4,b7b7764cfa1c523e4e93ab2a79a946c4,is home. <3,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,07/20/09 02:31 AM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1


In [ ]:
len(datos)

9917

## 2. Limpieza de los datos

Dejamos solo las columnas con las que vamos a trabajar.

In [6]:
datos_limpios = datos.drop(['DATE','NETWORKSIZE','BETWEENNESS','NBETWEENNESS','DENSITY','BROKERAGE','NBROKERAGE','TRANSITIVITY'],axis=1)

In [7]:
datos_limpios.head()

,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,cAGR,cCON,cOPN
0,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y
1,b7b7764cfa1c523e4e93ab2a79a946c4,is so sleepy it's not even funny that's she ca...,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y
2,b7b7764cfa1c523e4e93ab2a79a946c4,is sore and wants the knot of muscles at the b...,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y
3,b7b7764cfa1c523e4e93ab2a79a946c4,likes how the day sounds in this new song.,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y
4,b7b7764cfa1c523e4e93ab2a79a946c4,is home. <3,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y


In [8]:
print(datos_limpios['STATUS'])

0                             likes the sound of thunder.
1       is so sleepy it's not even funny that's she ca...
2       is sore and wants the knot of muscles at the b...
3              likes how the day sounds in this new song.
4                                             is home. <3
                              ...                        
9912                         little things give you away.
9913                          is wishing it was Saturday.
9914                      is studying hard for the G.R.E.
9915                                snipers get more head
9916    Last night was amazing! Not only did I see *PR...
Name: STATUS, Length: 9917, dtype: object


In [ ]:
print(len(datos))

9917


### Filtrar textos cortos

A continuación eliminamos los registros cuyos textos tengan 20 o menos palabras

In [9]:
datos_limpios['NumOfWords'] = [len(str(x).split(" ")) for x in datos_limpios['STATUS']] 

In [ ]:
datos_limpios = datos_limpios[datos_limpios.NumOfWords>20]

In [ ]:
datos_limpios

,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,cAGR,cCON,cOPN,NumOfWords
2,b7b7764cfa1c523e4e93ab2a79a946c4,is sore and wants the knot of muscles at the b...,2.65,3.00,3.15,3.25,4.40,n,y,n,n,y,26
8,b7b7764cfa1c523e4e93ab2a79a946c4,was about to finish a digital painting before ...,2.65,3.00,3.15,3.25,4.40,n,y,n,n,y,27
30,b7b7764cfa1c523e4e93ab2a79a946c4,has a watch that matches her glasses. She wish...,2.65,3.00,3.15,3.25,4.40,n,y,n,n,y,21
32,b7b7764cfa1c523e4e93ab2a79a946c4,can't sit on her futon because she knows as so...,2.65,3.00,3.15,3.25,4.40,n,y,n,n,y,26
33,b7b7764cfa1c523e4e93ab2a79a946c4,should spend less time writing chapters for th...,2.65,3.00,3.15,3.25,4.40,n,y,n,n,y,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9890,d2ee2871aa02698e66fdc72b2218cdd8,"Yay, finally got my computer set up at my hous...",2.75,3.25,2.50,4.00,4.65,n,y,n,y,y,21
9902,ea890531d3e0547166efde52d843099e,Just wanted to thank everyone for all the supp...,4.75,4.25,3.75,3.50,5.00,y,y,y,y,y,28
9905,ea890531d3e0547166efde52d843099e,you know when you work your @$$ off for a real...,4.75,4.25,3.75,3.50,5.00,y,y,y,y,y,27
9907,fbe5aa478508d1dc931427ade5d9e1b4,Oh well. Only two things to do when your team...,3.25,2.75,2.25,4.00,4.00,n,n,n,y,y,25


In [ ]:
print(datos_limpios.dtypes)


#AUTHID        object
STATUS         object
sEXT          float64
sNEU          float64
sAGR          float64
sCON          float64
sOPN          float64
cEXT           object
cNEU           object
cAGR           object
cCON           object
cOPN           object
NumOfWords      int64
dtype: object


In [ ]:
#Tenemos datos vacíos o nulos
datos_limpios.isnull().sum()

#AUTHID       0
STATUS        0
sEXT          0
sNEU          0
sAGR          0
sCON          0
sOPN          0
cEXT          0
cNEU          0
cAGR          0
cCON          0
cOPN          0
NumOfWords    0
dtype: int64

In [ ]:
#Usar en caso de tener valores vacíos o nulos.
datos_limpios["STATUS"].fillna("Sin texto", inplace = True)

### A continuación realizaremos los siguientes pasos:

1. Separar el texto en Tokens
2. Convertir palabras a minúscula
3. Expandir contracciones
4. Remover urls, correos, saltos de línea
5. Eliminar caracteres repetidos
6. Eliminar nuevas líneas y pestañas
7. Remover saltos de línea
8. Remover comillas simples
9. Eliminar comas " , "
10. Remover numeros
11. Remover Caracteres no alfanumericos
12. Eliminar guiones entre palabras 
13. Eliminar los guiones dobles y triples
14. Eliminar espacios en blanco (al principio, final y espacios dobles)
15. Eleminar stop words 
16. Realizar stemmin/Lematizacion  
17. Remover signos de puntuación
18. Destokenizar


In [ ]:
datos_a_lista = datos_limpios['STATUS'].values.tolist()
print(len(datos_a_lista))
datos_a_lista

2221


["is sore and wants the knot of muscles at the base of her neck to stop hurting. On the other hand, YAY I'M IN ILLINOIS! <3",
 'was about to finish a digital painting before her tablet went haywire. Is now contemplating the many ways she wishes to exact her revenge on faulty technology.',
 "has a watch that matches her glasses. She wishes she hadn't spazzed with glee when she saw it in the store.",
 "can't sit on her futon because she knows as soon as she does she'll lie down and pass out. Uncomfortable rolly desk chair it is. :(",
 "should spend less time writing chapters for the story she'll never finish, and try reading more books. Preferably, during the daytime, and not at 3:00 AM.",
 'is grateful she did most of her work during her wait for a ride home this afternoon and then during the Open House. Sleeeep, she beckons.',
 'has grown weary of drawing her paranoid, soldier//housewife Jynx character in comic form, and must thusly go to sleep to replenish her desire to work tirelessl

In [ ]:
CONTRACTION_MAP = {
"ain't": "is not",
"aren't": "are not",
"can't": "can not",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have",
}

def expandir_contracciones(data, contraction_mapping =  CONTRACTION_MAP):
    """ampliar las palabras abreviadas a la forma real.
        p.ej. don't ----> do not
    
        argumentos:
             input_text: "texto" de tipo "String".
         
        regreso:
             valor: Texto con forma expandida de palabras abreviadas.
        
        Ejemplo:
        Input : ain't, aren't, can't, cause, can't've
        Output :  is not, are not, cannot, because, cannot have 
     """

    #Convertir palabras a minúscula
    data=data.lower()

    # Tokenización de texto en tokens.
    lista_de_tokens = data.split(' ')

    # Comprobando si el token dado coincide con la clave y reemplazando la palabra con el valor de la clave.
    
    # Comprobar si palabra está en lidt_Of_tokens o no.
    for palabra in lista_de_tokens: 
        # Verifique si la palabra encontrada está en el diccionario "Mapa de contracción" o no como clave. 
         if palabra in CONTRACTION_MAP: 
                # Si palabra está presente tanto en el diccionario como en la lista_de_tokens, reemplace esa palabra con el valor clave.
                lista_de_tokens = [item.replace(palabra, CONTRACTION_MAP[palabra]) for item in lista_de_tokens]
                
    # Converting list of tokens to String.
    string_de_tokens = ' '.join(str(i) for i in lista_de_tokens) 
    return string_de_tokens

In [ ]:
temp1 = []
#Separando pd.Series a una lista
for i in range(len(datos_a_lista)):
    temp1.append(expandir_contracciones(datos_a_lista[i],CONTRACTION_MAP))
temp1

['is sore and wants the knot of muscles at the base of her neck to stop hurting. on the other hand, yay i am in illinois! <3',
 'was about to finish a digital painting before her tablet went haywire. is now contemplating the many ways she wishes to exact her revenge on faulty technology.',
 'has a watch that matches her glasses. she wishes she had not spazzed with glee when she saw it in the store.',
 'can not sit on her futon because she knows as soon as she does she will lie down and pass out. uncomfortable rolly desk chair it is. :(',
 'should spend less time writing chapters for the story she will never finish, and try reading more books. preferably, during the daytime, and not at 3:00 am.',
 'is grateful she did most of her work during her wait for a ride home this afternoon and then during the open house. sleeeep, she beckons.',
 'has grown weary of drawing her paranoid, soldier//housewife jynx character in comic form, and must thusly go to sleep to replenish her desire to work t

In [ ]:
def depurar_datos(data):
    
    #Remover URLs 
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    data = url_pattern.sub(r'', data)

    #Eliminar caracteres repetidos y limitando toda la repetición a dos caracteres. (english)  #Input : Realllllllllyyyyy / Greeeeaaaatttt #Output : Reallyy / Greeaatt
    Pattern_alpha = re.compile(r"([A-Za-z])\1{1,}", re.DOTALL)
    data = Pattern_alpha.sub(r"\1\1", data)

    # Remover correos
    data = re.sub('\S*@\S*\s?', '', data)

    #eliminar nuevas líneas y pestañas
    data = data.replace('\\n', ' ').replace('\t',' ').replace('\\', ' ').replace('. com', '.com')

    # Remover saltos de línea
    data = re.sub('\s+', ' ', data)

    # Remover comillas simples
    data = re.sub("\'", "", data)

    #eliminar comas " , "
    data=re.sub(",","",data)

    # Remover numeros
    data = re.sub(r'\d+', '', data)

    #Remover Caracteres especiales
    data = re.sub(r"[^a-zA-Z-á-é,í,ó,ú,ü,Á-É-Í-Ó-Ú-ñ]"," ",data)

    #Convertir palabras a minúscula (Español)
    data=data.lower()

    #eliminar guiones entre palabras cience-fiction >>> cience fiction
    data=re.sub("-"," ",data)

    #Eliminar los guiones dobles y triples
    data= re.sub("--"," ", data)
    data= re.sub("---"," ", data)

    #Eliminar los espacios en blanco al principio
    data= re.sub(r"^\s+", "", data)

    #Eliminar los espacios en blanco al final
    data= re.sub(r"\s+$", "", data)

    #remover espacios dobles
    data = " ".join(data.split())

    

    

    return data

In [ ]:
temp = []
#Separando pd.Series a una lista
for i in range(len(temp1)):
    temp.append(depurar_datos(temp1[i]))
temp

['is sore and wants the knot of muscles at the base of her neck to stop hurting on the other hand yay i am in illinois',
 'was about to finish a digital painting before her tablet went haywire is now contemplating the many ways she wishes to exact her revenge on faulty technology',
 'has a watch that matches her glasses she wishes she had not spazzed with glee when she saw it in the store',
 'can not sit on her futon because she knows as soon as she does she will lie down and pass out uncomfortable rolly desk chair it is',
 'should spend less time writing chapters for the story she will never finish and try reading more books preferably during the daytime and not at am',
 'is grateful she did most of her work during her wait for a ride home this afternoon and then during the open house sleep she beckons',
 'has grown weary of drawing her paranoid soldier housewife jynx character in comic form and must thusly go to sleep to replenish her desire to work tirelessly on a mere audition',
 '

In [ ]:
def sentencia_a_palabras(sentencias):
    for sentencia in sentencias:
        yield(gensim.utils.simple_preprocess(str(sentencia), deacc=True))  # deacc=True remueve puntuación
        

datos_palabras = list(sentencia_a_palabras(temp))



In [ ]:
print(datos_palabras)

[['is', 'sore', 'and', 'wants', 'the', 'knot', 'of', 'muscles', 'at', 'the', 'base', 'of', 'her', 'neck', 'to', 'stop', 'hurting', 'on', 'the', 'other', 'hand', 'yay', 'am', 'in', 'illinois'], ['was', 'about', 'to', 'finish', 'digital', 'painting', 'before', 'her', 'tablet', 'went', 'haywire', 'is', 'now', 'contemplating', 'the', 'many', 'ways', 'she', 'wishes', 'to', 'exact', 'her', 'revenge', 'on', 'faulty', 'technology'], ['has', 'watch', 'that', 'matches', 'her', 'glasses', 'she', 'wishes', 'she', 'had', 'not', 'spazzed', 'with', 'glee', 'when', 'she', 'saw', 'it', 'in', 'the', 'store'], ['can', 'not', 'sit', 'on', 'her', 'futon', 'because', 'she', 'knows', 'as', 'soon', 'as', 'she', 'does', 'she', 'will', 'lie', 'down', 'and', 'pass', 'out', 'uncomfortable', 'rolly', 'desk', 'chair', 'it', 'is'], ['should', 'spend', 'less', 'time', 'writing', 'chapters', 'for', 'the', 'story', 'she', 'will', 'never', 'finish', 'and', 'try', 'reading', 'more', 'books', 'preferably', 'during', 'the'

In [ ]:
for x in datos_palabras[:500]:
  print(x)

['is', 'sore', 'and', 'wants', 'the', 'knot', 'of', 'muscles', 'at', 'the', 'base', 'of', 'her', 'neck', 'to', 'stop', 'hurting', 'on', 'the', 'other', 'hand', 'yay', 'am', 'in', 'illinois']
['was', 'about', 'to', 'finish', 'digital', 'painting', 'before', 'her', 'tablet', 'went', 'haywire', 'is', 'now', 'contemplating', 'the', 'many', 'ways', 'she', 'wishes', 'to', 'exact', 'her', 'revenge', 'on', 'faulty', 'technology']
['has', 'watch', 'that', 'matches', 'her', 'glasses', 'she', 'wishes', 'she', 'had', 'not', 'spazzed', 'with', 'glee', 'when', 'she', 'saw', 'it', 'in', 'the', 'store']
['can', 'not', 'sit', 'on', 'her', 'futon', 'because', 'she', 'knows', 'as', 'soon', 'as', 'she', 'does', 'she', 'will', 'lie', 'down', 'and', 'pass', 'out', 'uncomfortable', 'rolly', 'desk', 'chair', 'it', 'is']
['should', 'spend', 'less', 'time', 'writing', 'chapters', 'for', 'the', 'story', 'she', 'will', 'never', 'finish', 'and', 'try', 'reading', 'more', 'books', 'preferably', 'during', 'the', 'da

In [ ]:
len(datos_palabras)

2221

## Lista de palabras Stopwords

In [ ]:
# Carga una lista de palabras stopwords desde nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
print(stop_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
{'those', "it's", 'does', 'shan', 'nor', 'needn', 'above', 'do', 'through', 'her', 'who', 'and', 'yourself', "mustn't", 'wasn', 'an', 'each', 'is', 'y', 'wouldn', 'd', 'their', 'to', 'yourselves', 'shouldn', "you've", 'were', 'about', 'here', 'it', 'until', 'not', 'with', 'they', 'at', 'yours', 'that', 'the', "shouldn't", "you'd", 'what', 'his', 'ourselves', 'herself', "you'll", 'haven', 'doesn', 't', 'against', 'ain', 'we', 'a', 'during', 'them', "weren't", 'don', 'between', 'ours', "aren't", "she's", 'theirs', "wouldn't", 'himself', 've', "needn't", 'or', 'doing', 'once', 'while', 's', 'will', 'if', 'she', 'your', 'there', 'couldn', 'whom', 'in', 'below', 'from', 'should', "didn't", 'being', 'as', "couldn't", 'only', 'him', 'same', 'because', 'before', 'o', 'themselves', 'most', 'now', 'other', "hasn't", 'weren', 'have', 'll', 'after', 'some', 'no', 'hers', 'just', 'of', 'wa

In [ ]:
#nuevas StopWords
new_stopwords = ['a', 'accuracy', 'ahora', 'ahí', 'algo', 'alguien', 'algun', 'alguna', 'algunas', 'alguno', 'algunos', 'algún', 'all', 'all', 'amazing', 'and', 'ante', 'antes', 'any', 'ape', 'apple', 'aquellos', 'aquél', 'aquí', 'así', 'aun', 'aún', 'b','c','d','f','g','h','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','background', 'bass', 'bate', 'bestia', 'big', 'bio', 'blogger', 'board', 'breef', 'buenardo', 'bull', 'bust', 'bycare', 'cagan', 'caiga', 'chafa', 'chilas', 'chilo', 'chilo', 'chilos', 'chingada', 'chingados', 'chingo', 'chingon', 'chingón', 'chorro', 'cinco', 'clover', 'cnc', 'como', 'con', 'contra', 'cual', 'cualquier', 'cualquiera', 'cuando', 'cuanta', 'cuanto', 'culiacán', 'curry', 'cuálcuales', 'cuáles', 'cuáles', 'cuándo', 'cuánta', 'cuánta', 'cuántas', 'cuánto', 'cuánto', 'cuántos', 'd', 'de', 'del', 'del', 'desde', 'después', 'di', 'disque', 'doce', 'donde', 'durante', 'dónde', 'e', 'eh', 'el', 'ella', 'ellas', 'ellos', 'en', 'enam', 'entonces', 'entre', 'es', 'esa', 'esas', 'ese', 'eso', 'esos', 'esta', 'estado', 'este', 'esto', 'estos', 'esté', 'ex', 'f1', 'facebook', 'factory', 'fantasy', 'far', 'farmear', 'farmeas', 'farmeo', 'fc', 'flawless', 'for', 'fuck', 'fx', 'game', 'genchi', 'giras', 'good', 'gto', 'guey', 'guy', 'güey', 'ha', 'hacia', 'hacía', 'han', 'hardy', 'has', 'hasta', 'he', 'hey', 'hiba', 'hillson', 'history', 'home', 'híjole', 'i', 'iba', 'ii', 'iii', 'increment', 'instagram', 'interative', 'ipad', 'iterative', 'iv', 'ix', 'java', 'javascript', 'jayeres', 'just', 'jutsu', 'kaizen', 'kills', 'kimtsu', 'king', 'la', 'ladder', 'lanza', 'las', 'le', 'legends', 'les', 'link', 'lo', 'lofi', 'lol', 'lootiando', 'los', 'lou', 'machín', 'making', 'mame', 'mas', 'match', 'mayoría', 'me', 'menos', 'metiches', 'mi', 'mis', 'misery', 'mlb', 'modo', 'mom', 'montón', 'mother', 'mu', 'mucha', 'muchas', 'mucho', 'muchos', 'muy', 'my', 'más', 'mí', 'mís', 'nada', 'name', 'naming', 'nba', 'ness', 'neta', 'new', 'ni', 'nintendo', 'nomas', 'nomás', 'nos', 'nosotras', 'nosotros', 'nuestra', 'nuestras', 'nuestro', 'nuestros', 'o', 'oda', 'of', 'off', 'oiga', 'ok', 'okey', 'one piece', 'oniska', 'os', 'otra', 'otras', 'otro', 'otros', 'oye', 'para', 'past', 'pbp', 'pedo', 'pen', 'pero', 'peros', 'perrones', 'perrones', 'perrón', 'pinche', 'placosón', 'play', 'plc', 'plus', 'poco', 'podcast', 'por', 'porque', 'post', 'predator', 'press', 'pro', 'profes', 'prota', 'psb', 'pues', 'puesto', 'puig', 'pushar', 'python', 'que', 'quien', 'quienes', 'quién', 'qué', 'rapidón', 'rb', 'recall', 'rever', 'rex', 'riqui', 'robotcidio', 'rocket', 'rpg', 'runes', 's', 'sandbox', 'schering', 'se', 'season', 'shooters', 'si', 'sido', 'siempre', 'simón', 'sin', 'sinaloense', 'sino', 'six', 'skincare', 'sloth', 'smurfs', 'sobre', 'solo', 'stephen', 'streamers', 'su', 'sub', 'sugar', 'sus', 'sé', 'sí', 'sólo', 'tal', 'también', 'tanto', 'te', 'team', 'tebas', 'tec', 'tenida', 'tenidas', 'tenido', 'tenidos', 'terco', 'the', 'thor', 'ti', 'tienden', 'tipo', 'to', 'toda', 'todas', 'todo', 'todos', 'traka', 'trash', 'través', 'tricast', 'tu', 'tuaf', 'tuneado', 'tus', 'tuya', 'tuyas', 'tuyo', 'tuyos', 'té', 'tí', 'tú', 'tú', 'u', 'udemy', 'uff', 'un', 'una', 'unas', 'unas', 'uno', 'unos', 'urbania', 'v', 'v', 'va', 'valorant', 'verga', 'vi', 'vii', 'viii', 'vosotras', 'vosotros', 'walkies', 'wars', 'watch', 'wookies', 'x', 'xdray', 'y', 'ya', 'yaiba', 'yo', 'youtube', 'youtuber', 'youtubers', 'yoyo', 'zelda', 'zoom', 'ápex', 'él', 'ésta', 'éstas', 'éste', 'ésto', 'éstos', 'ó']
new_stopwords.sort()
print(new_stopwords)


['a', 'accuracy', 'ahora', 'ahí', 'algo', 'alguien', 'algun', 'alguna', 'algunas', 'alguno', 'algunos', 'algún', 'all', 'all', 'amazing', 'and', 'ante', 'antes', 'any', 'ape', 'apple', 'aquellos', 'aquél', 'aquí', 'así', 'aun', 'aún', 'b', 'background', 'bass', 'bate', 'bestia', 'big', 'bio', 'blogger', 'board', 'breef', 'buenardo', 'bull', 'bust', 'bycare', 'c', 'cagan', 'caiga', 'chafa', 'chilas', 'chilo', 'chilo', 'chilos', 'chingada', 'chingados', 'chingo', 'chingon', 'chingón', 'chorro', 'cinco', 'clover', 'cnc', 'como', 'con', 'contra', 'cual', 'cualquier', 'cualquiera', 'cuando', 'cuanta', 'cuanto', 'culiacán', 'curry', 'cuálcuales', 'cuáles', 'cuáles', 'cuándo', 'cuánta', 'cuánta', 'cuántas', 'cuánto', 'cuánto', 'cuántos', 'd', 'd', 'de', 'del', 'del', 'desde', 'después', 'di', 'disque', 'doce', 'donde', 'durante', 'dónde', 'e', 'eh', 'el', 'ella', 'ellas', 'ellos', 'en', 'enam', 'entonces', 'entre', 'es', 'esa', 'esas', 'ese', 'eso', 'esos', 'esta', 'estado', 'este', 'esto', '

## Remover stopwords 

In [ ]:
def remover_stopword(lista_texto):
  sin_stopwords = [palabra for palabra in lista_texto if not palabra in stop_words]
  return sin_stopwords

In [ ]:
stopword_text=[]
for i in range(len(datos_palabras)):
  stopword_text.append(remover_stopword(datos_palabras[i]))    


In [ ]:
for x in stopword_text:
  print(x)

['sore', 'wants', 'knot', 'muscles', 'base', 'neck', 'stop', 'hurting', 'hand', 'yay', 'illinois']
['finish', 'digital', 'painting', 'tablet', 'went', 'haywire', 'contemplating', 'many', 'ways', 'wishes', 'exact', 'revenge', 'faulty', 'technology']
['watch', 'matches', 'glasses', 'wishes', 'spazzed', 'glee', 'saw', 'store']
['sit', 'futon', 'knows', 'soon', 'lie', 'pass', 'uncomfortable', 'rolly', 'desk', 'chair']
['spend', 'less', 'time', 'writing', 'chapters', 'story', 'never', 'finish', 'try', 'reading', 'books', 'preferably', 'daytime']
['grateful', 'work', 'wait', 'ride', 'home', 'afternoon', 'open', 'house', 'sleep', 'beckons']
['grown', 'weary', 'drawing', 'paranoid', 'soldier', 'housewife', 'jynx', 'character', 'comic', 'form', 'must', 'thusly', 'go', 'sleep', 'replenish', 'desire', 'work', 'tirelessly', 'mere', 'audition']
['got', 'taught', 'use', 'register', 'enjoyed', 'days', 'summer', 'looking', 'forward', 'going', 'beach', 'monday']
['prepared', 'weekend', 'homeowrk', 'mor

## Lematización

In [ ]:
!python -m spacy download en

     |████████████████████████████████| 12.0 MB 7.5 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()


In [ ]:
def realizar_lematizacion(lista):
    texto = " ".join(lista)
    doc = nlp(texto)
    lemms = []
    for token in doc:
        lemms.append(token.lemma_)
    return lemms

In [ ]:
texto_lematizado=[]
for i in range(len(stopword_text)):
  texto_lematizado.append(realizar_lematizacion(stopword_text[i])) 

In [ ]:
print(texto_lematizado)

[['sore', 'want', 'knot', 'muscle', 'base', 'neck', 'stop', 'hurt', 'hand', 'yay', 'illinois'], ['finish', 'digital', 'painting', 'tablet', 'go', 'haywire', 'contemplate', 'many', 'way', 'wish', 'exact', 'revenge', 'faulty', 'technology'], ['watch', 'match', 'glass', 'wish', 'spazze', 'glee', 'saw', 'store'], ['sit', 'futon', 'know', 'soon', 'lie', 'pass', 'uncomfortable', 'rolly', 'desk', 'chair'], ['spend', 'less', 'time', 'writing', 'chapter', 'story', 'never', 'finish', 'try', 'read', 'book', 'preferably', 'daytime'], ['grateful', 'work', 'wait', 'ride', 'home', 'afternoon', 'open', 'house', 'sleep', 'beckon'], ['grow', 'weary', 'drawing', 'paranoid', 'soldier', 'housewife', 'jynx', 'character', 'comic', 'form', 'must', 'thusly', 'go', 'sleep', 'replenish', 'desire', 'work', 'tirelessly', 'mere', 'audition'], ['get', 'teach', 'use', 'register', 'enjoy', 'day', 'summer', 'look', 'forward', 'go', 'beach', 'monday'], ['prepared', 'weekend', 'homeowrk', 'morning', 'due', 'realize', 'le

In [ ]:
print(stopword_text)

[['sore', 'wants', 'knot', 'muscles', 'base', 'neck', 'stop', 'hurting', 'hand', 'yay', 'illinois'], ['finish', 'digital', 'painting', 'tablet', 'went', 'haywire', 'contemplating', 'many', 'ways', 'wishes', 'exact', 'revenge', 'faulty', 'technology'], ['watch', 'matches', 'glasses', 'wishes', 'spazzed', 'glee', 'saw', 'store'], ['sit', 'futon', 'knows', 'soon', 'lie', 'pass', 'uncomfortable', 'rolly', 'desk', 'chair'], ['spend', 'less', 'time', 'writing', 'chapters', 'story', 'never', 'finish', 'try', 'reading', 'books', 'preferably', 'daytime'], ['grateful', 'work', 'wait', 'ride', 'home', 'afternoon', 'open', 'house', 'sleep', 'beckons'], ['grown', 'weary', 'drawing', 'paranoid', 'soldier', 'housewife', 'jynx', 'character', 'comic', 'form', 'must', 'thusly', 'go', 'sleep', 'replenish', 'desire', 'work', 'tirelessly', 'mere', 'audition'], ['got', 'taught', 'use', 'register', 'enjoyed', 'days', 'summer', 'looking', 'forward', 'going', 'beach', 'monday'], ['prepared', 'weekend', 'homeow

## Eliminar signos de puntuación Lematización



In [ ]:
lem_signos_puntuacion=[]
for x in texto_lematizado:
  texto = " ".join(x)
  lem_signos_puntuacion.append(texto)


In [ ]:
print(lem_signos_puntuacion)

['sore want knot muscle base neck stop hurt hand yay illinois', 'finish digital painting tablet go haywire contemplate many way wish exact revenge faulty technology', 'watch match glass wish spazze glee saw store', 'sit futon know soon lie pass uncomfortable rolly desk chair', 'spend less time writing chapter story never finish try read book preferably daytime', 'grateful work wait ride home afternoon open house sleep beckon', 'grow weary drawing paranoid soldier housewife jynx character comic form must thusly go sleep replenish desire work tirelessly mere audition', 'get teach use register enjoy day summer look forward go beach monday', 'prepared weekend homeowrk morning due realize leave necessary binder school ff', 'hate well find congratulation worth thing manage break far heart brave murder ever care please proceed andorid hell', 'late ride dad airport pick mamacita pm land illinois', 'complete love blackberry mom surprise yes excited cap completely necessary', 'need stop listen n

In [ ]:
def sentencia_a_palabras(sentencias):
    for sentencia in sentencias:
        yield(gensim.utils.simple_preprocess(str(sentencia), deacc=True))  # deacc=True remueve puntuación
        

datos_tokens_lem = list(sentencia_a_palabras(lem_signos_puntuacion))



In [ ]:
print(len(datos_tokens_lem))
print(datos_tokens_lem)


2221
[['sore', 'want', 'knot', 'muscle', 'base', 'neck', 'stop', 'hurt', 'hand', 'yay', 'illinois'], ['finish', 'digital', 'painting', 'tablet', 'go', 'haywire', 'contemplate', 'many', 'way', 'wish', 'exact', 'revenge', 'faulty', 'technology'], ['watch', 'match', 'glass', 'wish', 'spazze', 'glee', 'saw', 'store'], ['sit', 'futon', 'know', 'soon', 'lie', 'pass', 'uncomfortable', 'rolly', 'desk', 'chair'], ['spend', 'less', 'time', 'writing', 'chapter', 'story', 'never', 'finish', 'try', 'read', 'book', 'preferably', 'daytime'], ['grateful', 'work', 'wait', 'ride', 'home', 'afternoon', 'open', 'house', 'sleep', 'beckon'], ['grow', 'weary', 'drawing', 'paranoid', 'soldier', 'housewife', 'jynx', 'character', 'comic', 'form', 'must', 'thusly', 'go', 'sleep', 'replenish', 'desire', 'work', 'tirelessly', 'mere', 'audition'], ['get', 'teach', 'use', 'register', 'enjoy', 'day', 'summer', 'look', 'forward', 'go', 'beach', 'monday'], ['prepared', 'weekend', 'homeowrk', 'morning', 'due', 'realize'

## Unión de tokens

In [ ]:
def destokenizar(texto):
    return TreebankWordDetokenizer().detokenize(texto)

### Destokenización de Texto Lematizado

In [ ]:
new_data = []
for i in range(len(datos_tokens_lem)): 
    new_data.append(destokenizar(datos_tokens_lem[i]))

## Texto procesado

In [ ]:
new_data = np.array(new_data)
print(type(new_data))
print(new_data)

<class 'numpy.ndarray'>
['sore want knot muscle base neck stop hurt hand yay illinois'
 'finish digital painting tablet go haywire contemplate many way wish exact revenge faulty technology'
 'watch match glass wish spazze glee saw store' ...
 'know work really long time finally one day pay today day'
 'oh well two thing team get crush playoff happen booze handy'
 'last night amazing see propname propname last performance rent also end night fire lounge whole cast celebrate']


### Codificación de las etiquetas

Como el conjunto de datos es categórico, necesitamos convertir las etiquetas de personalidad de Neutral, No y Sí a un tipo float que nuestro modelo pueda entender. Para lograr esta tarea, implementaremos el método to_categorical de Keras.

In [ ]:
etiquetas = np.array(datos_limpios['cOPN'])
y = []
for i in range(len(etiquetas)):
    if etiquetas[i] == 'n':
        y.append(0)
    if etiquetas[i] == 'y':
        y.append(1)
y = np.array(y)
etiquetas = tf.keras.utils.to_categorical(y, 2, dtype="float32")
del y

In [ ]:
len(etiquetas)

2221

In [ ]:
print(np.array(datos_limpios['cOPN'][102:220]))

['y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y'
 'y' 'n' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y'
 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'n' 'n' 'n' 'n' 'y' 'y' 'y'
 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y'
 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y'
 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y'
 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y' 'y']


In [ ]:
print(etiquetas[102:220])

[[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 

### Secuenciado de datos y sepación en matrices

Implementaremos el tokenizador de Keras así como su método pad_sequences para transformar nuestros datos de texto en datos flotantes 3D, de lo contrario nuestras redes neuronales no podrán ser entrenadas con ellos.

In [ ]:
from keras.models import Sequential
from keras import layers
#from keras.optimizers import RMSprop, Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
#max_words = 5000
#max_len = 200

#tokenizer = Tokenizer(num_words=max_words)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(new_data)
word_index = tokenizer.word_index
total_unique_words = len(tokenizer.word_index) + 1 
sequences = tokenizer.texts_to_sequences(new_data)
max_seq_length = max([len(x) for x in sequences])
#textos = pad_sequences(sequences, maxlen=max_len)
textos = pad_sequences(sequences, maxlen=max_seq_length)
#print(textos[100:110])
max_words = total_unique_words
max_len = max_seq_length

print(textos[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0  496   15 3374 1692  953 1327
  159  271  272  212 2279]


In [ ]:
print(new_data[110])

another solid show dtcc love propname propname mo absence funny buddy also take stage


In [ ]:
print("Palabras únicas: ",total_unique_words)
print("Longitud máxima de texto: ",max_seq_length)

Palabras únicas:  7936
Longitud máxima de texto:  61


In [ ]:
print(textos)
print(etiquetas)

[[   0    0    0 ...  272  212 2279]
 [   0    0    0 ...  667 3377 2283]
 [   0    0    0 ...  668 1693  353]
 ...
 [   0    0    0 ...  211   13    6]
 [   0    0    0 ...  205 7935 1190]
 [   0    0    0 ...  208  588  361]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


### Dividimos el dataset

In [ ]:
#Dividimos el dataset en entrenamiento y validación 75% y 25%
X_train, X_test, y_train, y_test = train_test_split(textos,etiquetas, test_size=0.25, random_state=0)
print(len(X_train),len(X_test))
print(len(y_train),len(y_test))

1665 556
1665 556


## Construcción del modelo

### Perceptrón Multicapa

---



In [ ]:
model = Sequential()
model.add(layers.Embedding(max_words, 128, input_length=max_len)) 
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))  #model.add(layers.Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

earlyStopping = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1, mode='min')
mcp = ModelCheckpoint("best_model0.hdf5", monitor="val_accuracy", save_best_only=True, save_weights_only=False, verbose=1)
model_history = model.fit(X_train, y_train, verbose=1,
                          validation_data = (X_test, y_test), epochs=10,
                          callbacks=[earlyStopping,mcp], batch_size= 64, shuffle=True)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 61, 128)           1015808   
                                                                 
 flatten (Flatten)           (None, 7808)              0         
                                                                 
 dense (Dense)               (None, 512)               3998208   
                                                                 
 dense_1 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 5,015,042
Trainable params: 5,015,042
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
26/27 [===========================>..] - ETA: 0s - loss: 0.6077 - accuracy: 0.7121
Epoch 1: val_accuracy improved from -inf to 0.72302, saving model to best_model0.hdf

### LSTM Sencilla

In [ ]:
model1 = Sequential()
model1.add(layers.Embedding(max_words, 20))
model1.add(layers.LSTM(15,dropout=0.5))
model1.add(layers.Dense(2,activation='softmax'))


model1.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])
#Implementing model checkpoins to save the best metric and do not lose it on training.
checkpoint1 = ModelCheckpoint("best_model1.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', save_freq='epoch',save_weights_only=False)
history = model1.fit(X_train, y_train, epochs=50,validation_data=(X_test, y_test),callbacks=[checkpoint1])

Epoch 1/50
53/53 [==============================] - ETA: 0s - loss: 0.6165 - accuracy: 0.7105
Epoch 1: val_accuracy improved from -inf to 0.72302, saving model to best_model1.hdf5
53/53 [==============================] - 10s 99ms/step - loss: 0.6165 - accuracy: 0.7105 - val_loss: 0.6047 - val_accuracy: 0.7230
Epoch 2/50
53/53 [==============================] - ETA: 0s - loss: 0.5857 - accuracy: 0.7159
Epoch 2: val_accuracy did not improve from 0.72302
53/53 [==============================] - 3s 63ms/step - loss: 0.5857 - accuracy: 0.7159 - val_loss: 0.5998 - val_accuracy: 0.7230
Epoch 3/50
52/53 [============================>.] - ETA: 0s - loss: 0.5633 - accuracy: 0.7157
Epoch 3: val_accuracy did not improve from 0.72302
53/53 [==============================] - 3s 53ms/step - loss: 0.5632 - accuracy: 0.7159 - val_loss: 0.5750 - val_accuracy: 0.7230
Epoch 4/50
53/53 [==============================] - ETA: 0s - loss: 0.5119 - accuracy: 0.7195
Epoch 4: val_accuracy did not improve from 0.

### RNN Simple

In [ ]:
#model0 = Sequential()
#model0.add(layers.Embedding(max_words, 15))
#model0.add(layers.SimpleRNN(15))
#model0.add(layers.Dense(3,activation='softmax'))


#model0.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])
#Implementing model checkpoins to save the best metric and do not lose it on training.
#checkpoint0 = ModelCheckpoint("best_model0.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
#history = model0.fit(X_train, y_train, epochs=5,validation_data=(X_test, y_test),callbacks=[checkpoint0])

### LSTM Bidireccional

In [ ]:
model2 = Sequential()
model2.add(layers.Embedding(max_words, 40, input_length=max_len))
model2.add(layers.Bidirectional(layers.LSTM(20,dropout=0.6)))
model2.add(layers.Dense(2,activation='softmax'))
model2.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])
#Implementing model checkpoins to save the best metric and do not lose it on training.
checkpoint2 = ModelCheckpoint("best_model2.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model2.fit(X_train, y_train, epochs=70,validation_data=(X_test, y_test),callbacks=[checkpoint2])

Epoch 1/70
53/53 [==============================] - ETA: 0s - loss: 0.6071 - accuracy: 0.7045
Epoch 1: val_accuracy improved from -inf to 0.72302, saving model to best_model2.hdf5
53/53 [==============================] - 10s 112ms/step - loss: 0.6071 - accuracy: 0.7045 - val_loss: 0.6024 - val_accuracy: 0.7230
Epoch 2/70
53/53 [==============================] - ETA: 0s - loss: 0.5834 - accuracy: 0.7159
Epoch 2: val_accuracy did not improve from 0.72302
53/53 [==============================] - 3s 49ms/step - loss: 0.5834 - accuracy: 0.7159 - val_loss: 0.5897 - val_accuracy: 0.7230
Epoch 3/70
52/53 [============================>.] - ETA: 0s - loss: 0.5552 - accuracy: 0.7163
Epoch 3: val_accuracy did not improve from 0.72302
53/53 [==============================] - 2s 46ms/step - loss: 0.5555 - accuracy: 0.7159 - val_loss: 0.5892 - val_accuracy: 0.7230
Epoch 4/70
52/53 [============================>.] - ETA: 0s - loss: 0.4894 - accuracy: 0.7416
Epoch 4: val_accuracy improved from 0.72302 

### Modelo Convolucional 1D

 Según la teoría tiende a sobreajustarse extremadamente rápido en conjuntos de datos pequeños. 

In [ ]:
from keras import regularizers
model3 = Sequential()
model3.add(layers.Embedding(max_words, 40, input_length=max_len))
model3.add(layers.Conv1D(20, 6, activation='relu',kernel_regularizer=regularizers.l1_l2(l1=2e-3, l2=2e-3),bias_regularizer=regularizers.l2(2e-3)))
model3.add(layers.MaxPooling1D(5))
model3.add(layers.Conv1D(20, 6, activation='relu',kernel_regularizer=regularizers.l1_l2(l1=2e-3, l2=2e-3),bias_regularizer=regularizers.l2(2e-3)))
model3.add(layers.GlobalMaxPooling1D())
model3.add(layers.Dense(2,activation='softmax'))
model3.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['acc'])
metric = 'val_acc'
#checkpoint3 = ModelCheckpoint("best_model3.hdf5", monitor=metric, verbose=2,save_best_only=True, mode='auto', period=1,save_weights_only=False)
checkpoint3 = ModelCheckpoint("best_model3.hdf5", monitor=metric, verbose=2,save_best_only=True, mode='auto',save_weights_only=False)

history = model3.fit(X_train, y_train, epochs=100,validation_data=(X_test, y_test),callbacks=[checkpoint3])

#metric = 'val_accuracy'
#ModelCheckpoint(filepath=r"C:\Users\reda.elhail\Desktop\checkpoints\{}".format(Name), monitor=metric,
                    #verbose=2, save_best_only=True, mode='max')]

Epoch 1/100
52/53 [============================>.] - ETA: 0s - loss: 1.3112 - acc: 0.6923
Epoch 1: val_acc improved from -inf to 0.72302, saving model to best_model3.hdf5
53/53 [==============================] - 2s 21ms/step - loss: 1.3109 - acc: 0.6925 - val_loss: 0.9687 - val_acc: 0.7230
Epoch 2/100
52/53 [============================>.] - ETA: 0s - loss: 0.8132 - acc: 0.7157
Epoch 2: val_acc did not improve from 0.72302
53/53 [==============================] - 1s 10ms/step - loss: 0.8130 - acc: 0.7159 - val_loss: 0.6847 - val_acc: 0.7230
Epoch 3/100
50/53 [===========================>..] - ETA: 0s - loss: 0.6526 - acc: 0.7144
Epoch 3: val_acc did not improve from 0.72302
53/53 [==============================] - 1s 10ms/step - loss: 0.6504 - acc: 0.7159 - val_loss: 0.6380 - val_acc: 0.7230
Epoch 4/100
52/53 [============================>.] - ETA: 0s - loss: 0.6303 - acc: 0.7157
Epoch 4: val_acc did not improve from 0.72302
53/53 [==============================] - 1s 11ms/step - loss:

### Validación de los modelos

In [ ]:
#Cargamos uno de los modelos
#best_model = keras.models.load_model("best_model2.hdf5")
best_model = keras.models.load_model("best_model1.hdf5")
#best_model3.hdf5

In [ ]:
test_loss, test_acc = best_model.evaluate(X_test, y_test, verbose=2)
print('Model accuracy: ',test_acc)

18/18 - 1s - loss: 0.6047 - accuracy: 0.7230 - 643ms/epoch - 36ms/step
Model accuracy:  0.7230215668678284


In [ ]:
predictions = best_model.predict(X_test)

In [ ]:
print(predictions)
print(y_test)

[[0.2205816  0.7794184 ]
 [0.18660775 0.8133923 ]
 [0.21042977 0.7895702 ]
 ...
 [0.21661064 0.78338933]
 [0.19566453 0.8043355 ]
 [0.18889587 0.8111042 ]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [0. 1.]
 [0. 1.]]


###  Probando uno de los modelos

In [ ]:
clases = ['No','Yes']

In [ ]:
# Ej Correctos: 62 = No, 101 = Neutro; 10, 30, 45, 171,212 = Sí; 
# Ej Erróneos: 60
indice_prueba = 62
texto_prueba = datos_limpios["texto"][indice_prueba] 
print(texto_prueba)
print(datos_limpios["presenta_apertura"][indice_prueba])

In [ ]:
sequence = tokenizer.texts_to_sequences([texto_prueba])
test = pad_sequences(sequence, maxlen=max_len)
clases[np.around(best_model.predict(test), decimals=0).argmax(axis=1)[0]]

### Matriz de Confusión
Revisaremos la matriz de confusión, para entender mejor la capacidad de clasificación y generalización. Vamos a trazarla.

In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test.argmax(axis=1), np.around(predictions, decimals=0).argmax(axis=1))

In [ ]:
import seaborn as sns
conf_matrix = pd.DataFrame(matrix, index = ['Neutro','No','Sí'],columns = ['Neutro','No','Sí'])
#Normalizando
conf_matrix = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]
plt.figure(figsize = (15,15))
sns.heatmap(conf_matrix, annot=True, annot_kws={"size": 15})